## Reddit network of user interactions for suspicious accounts

1. Import Reddit usernames (https://www.reddit.com/wiki/suspiciousaccounts)
2. For each user, identify the threads they have commented on
3. For each of the threads, extract the reply/comment tree into csv (sender username, receiver username, time) 

In [1]:
import praw
import csv

import networkx as nx
G=nx.Graph()

import matplotlib.pyplot as plt

# Authenticate into Reddit
reddit = praw.Reddit(client_id='1QKQYY6zEUgYUw',
                     client_secret='8xyLf9B_raDH9FIs54t3yzYqBqc',
                     password='redditpw',
                     user_agent='testscript by /u/praw_test_1',
                     username='praw_test_1')

In [21]:
def csv_reply_network():
    
    
    with open('../Data_Suspicious_Users/comments.csv', mode='r') as commentsreader:
        commentsreader = csv.DictReader(commentsreader)
        
        for row in commentsreader:         
            submission_id = row['link_id'].split("_")[1]
            time_utc = row['created_utc']
            
            with open('Data_reply/'+ row['author.name'] +'_'+ submission_id +'.csv', mode='a') as comments_writer:
                header = ['user_name', 'parent_name', 'time']
                
                comments_writer = csv.writer(comments_writer, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                comments_writer.writerow(i for i in header)
                
                submission = reddit.submission(submission_id)
                submission.comments.replace_more(limit=None)
                
                for comment in submission.comments.list():

                    if comment.is_root is False:
                        parent = comment.parent()

                        comments_writer.writerow([comment.author, parent.author, time_utc])

                        for reply in comment.replies.list():
                            reply_parent = reply.parent()

                            comments_writer.writerow([reply.author, reply_parent.author, reply.created_utc])

            # Remove duplicate comments/replies
            with open('Data_reply/'+row['author.name']+'_'+ submission_id +'.csv', 'r') as in_file, open('Data_reply/'+row['author.name']+'_'+ submission_id +'_noroot.csv','w') as out_file:
                seen = set() # set for fast O(1) amortized lookup
                for line in in_file:
                    if line in seen: continue # skip duplicate

                    seen.add(line)
                    out_file.write(line)

csv_reply_network()


KeyboardInterrupt: 